In [144]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
import pickle
import matplotlib.pyplot as plt
import gzip
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [145]:
file = gzip.open("./dataset.pickle",'rb')
x_train, y_train, x_test, y_test = pickle.load(file)
file.close()

In [146]:
 x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, shuffle=True)

In [147]:
print("Tamanhos:", len(x_train), len(x_val), len(x_test))
print("Formato:", x_train.shape)
print("Balanceamento ->", "Classe 0:", len(y_train[y_train==0]), 
                          "Classe 1:", len(y_train[y_train==1]), 
                          "Classe 2:", len(y_train[y_train==2]))

Tamanhos: 13204 1468 3669
Formato: (13204, 64, 64)
Balanceamento -> Classe 0: 133 Classe 1: 7205 Classe 2: 5866


In [148]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train.reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
y_val = enc.transform(y_val.reshape(-1, 1)).toarray()

In [149]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_val = np.expand_dims(x_val, -1)

In [150]:
train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val = tf.data.Dataset.from_tensor_slices((x_val, y_val))

In [151]:
train = train.batch(32)
test = test.batch(32)
val = val.batch(32)

In [128]:
model = models.Sequential()
model.add(layers.Input((None, None, 1)))
model.add(layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding="same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="same"))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, None, None, 32)    320       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, None, None, 32)    0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, None, None, 64)    18496     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, None, None, 64)    0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, None, None, 128)   73856     
_________________________________________________________________
global_average_pooling2d_14  (None, 128)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)             

In [130]:
epochs=10
history = model.fit(train, epochs=epochs, validation_data=val)

Epoch 1/10
413/413 [==============================] - 14s 34ms/step - loss: 0.3208 - accuracy: 0.8737 - val_loss: 0.0744 - val_accuracy: 0.9748
Epoch 2/10
413/413 [==============================] - 16s 39ms/step - loss: 0.0457 - accuracy: 0.9865 - val_loss: 0.0344 - val_accuracy: 0.9898
Epoch 3/10
413/413 [==============================] - 19s 46ms/step - loss: 0.0447 - accuracy: 0.9846 - val_loss: 0.0453 - val_accuracy: 0.9877
Epoch 4/10
413/413 [==============================] - 19s 46ms/step - loss: 0.0200 - accuracy: 0.9936 - val_loss: 0.0177 - val_accuracy: 0.9959
Epoch 5/10
413/413 [==============================] - 19s 46ms/step - loss: 0.0153 - accuracy: 0.9951 - val_loss: 0.0144 - val_accuracy: 0.9959
Epoch 6/10
413/413 [==============================] - 19s 46ms/step - loss: 0.0144 - accuracy: 0.9955 - val_loss: 0.0058 - val_accuracy: 0.9980
Epoch 7/10
413/413 [==============================] - 19s 46ms/step - loss: 0.0106 - accuracy: 0.9964 - val_loss: 0.0115 - val_accuracy:

In [165]:
predictions = model.evaluate(test)

115/115 [==============================] - 1s 12ms/step - loss: 0.0072 - accuracy: 0.9978
